#### 

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
import cvxpy as cp
from scipy.optimize import linprog
from scripts.estimation import *
from scripts.utils import *

In [6]:
rand_seed = 1
num_position = 5
bike_num = 20
lambd = 10
grid_size = 10
beta0 = 1
beta1_true = -1
T = 100
loc_bound = 5

In [10]:
np.random.seed(seed=rand_seed)
posx = np.random.uniform(-4*loc_bound/5,4*loc_bound/5,num_position)
posy = np.random.uniform(-4*loc_bound/5,4*loc_bound/5,num_position)

array([-0.66382396,  1.76259595, -3.999085  , -1.58133942, -2.82595287])